In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import cufflinks
import plotly as py
import pylab as plt
import sys
sys.path.append(r'/nfs/iil/disks/hip_ana_sim_01/dgottesm/analysis_and_tools/analysisPP/brk_gen1/Automation/')

from PostProcess.sine_analysis.inl_dnl import extract_inl_dnl
from PostProcess.global_functions.generate_data import *
from PostProcess.sine_analysis.other_statistical_analysis import zero_one_dist_at_hist

from ipywidgets import interact
import ipywidgets as widgets

In [10]:
slicer_max_offset_mv=10

noise_std_mv=widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='noise_std_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
offset_volt_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
slicer1_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='slicer1_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
slicer2_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='slicer2_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))

dac0_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='dac0_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
dac1_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='dac1_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
dac2_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='dac2_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
dac3_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='dac3_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))
dac4_offset_mv=widgets.FloatSlider(value=0, min=-slicer_max_offset_mv, max=slicer_max_offset_mv, step=0.1, description='dac4_offset_mv:', continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f', layout=widgets.Layout(width='95%'), style=dict(description_width='initial'))

dac_col=[f'reads_read_{i}_' for i in range(1,6)]

@interact(noise_std_mv=noise_std_mv, slicer1_offset=slicer1_offset_mv,slicer2_offset=slicer2_offset_mv,
           dac0_offset_mv=dac0_offset_mv,dac1_offset_mv=dac1_offset_mv,dac2_offset_mv=dac2_offset_mv,dac3_offset_mv=dac3_offset_mv,dac4_offset_mv=dac4_offset_mv, 
           plot_hist_after_each_dac=False, 
           input_type=dict(sine=DataPattern.sine, pam2=DataPattern.PRBS_pam2, pam4=DataPattern.PRBS_pam4, DC=DataPattern.DC),
           offset_volt_mv=offset_volt_mv,
         )
def plotme(noise_std_mv, slicer1_offset,slicer2_offset,
           dac0_offset_mv,dac1_offset_mv,dac2_offset_mv,dac3_offset_mv,dac4_offset_mv, 
           plot_hist_after_each_dac, 
           input_type,offset_volt_mv):
    tau_ns=0
    noise_std=noise_std_mv*1e-3
    offset_ns=0
    offset_volt=offset_volt_mv*1e-3
    slicers_offset=np.array([slicer1_offset,slicer2_offset]*3)*1e-3
    dac_offset=np.array([dac0_offset_mv,dac1_offset_mv,dac2_offset_mv,dac3_offset_mv,dac4_offset_mv,0])*1e-3
    slicers_offset+=dac_offset
    df = SimulateSingleSar(noise_std_volt=noise_std, 
                           tau_ns=tau_ns, 
                           pattern_type=input_type,
                           offset_ns=offset_ns,
                           offset_volt=offset_volt,
                           slicers_offset=slicers_offset
                          ).simulate_raw_data()
    hist=(df.clipped.value_counts().sort_index()+pd.Series(0,index=range(64))).fillna(0)
    hist.rename_axis(index='code').\
            plot(kind='bar', title='hist of raw data', figsize=[20,5], grid=True)
    plt.figure()
    zero_one_dist_at_hist(hist).rename_axis(index='bit').plot(grid=True, figsize=(20, 5), title='1/0 ratio per bit')
    plt.figure()
    inl_dnl=extract_inl_dnl(lst=hist, number_of_bits=6, plot=False, truncate_tails=10, sine=True, histogram_input=True)
    
    fig, ax_dnl = plt.subplots(1, 2, figsize=(20, 5)) 
    inl_dnl.dnl.sort_index().plot(title='dnl', ax=ax_dnl[0])
    inl_dnl.inl.sort_index().plot(title='inl', ax=ax_dnl[1])

    if plot_hist_after_each_dac:
        fig, ax = plt.subplots(3, 2, figsize=(20, 10)) 
        ax=ax.flatten().tolist()[::-1]
        for col in dac_col:
            df[col].to_frame(col.replace('reads_read','input of dac ')).plot.hist(bins=50, ax=ax.pop(), grid=True)
    return None

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='noise_std_mv:', layout=Layo…

In [3]:
DataPattern.PRBS_pam2

0